# Calculating BII at the Ethnologue Polygon Level

In [18]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

import mapclassify

from rapidfuzz import process, fuzz

from shapely.geometry import Point

import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats

In [19]:
# Set base project path
base_path = Path("C:/Users/juami/Dropbox/RAships/2-Folklore-Nathan-Project/EA-Maps-Nathan-project/Measures_work")

# Set file paths
poscol_path = base_path / "data" / "raw" / "ethnologue" / "ancestral_characteristics_database_language_level" / "Ethnologue_16_shapefile" / "langa_no_overlap_biggest_clean.shp"

data_path = base_path / "data" / "interim"
maps_path = base_path / "maps" / "raw"
bii_path = maps_path / "BII" / "bii-2020_v2-1-1.tif"

In [20]:
# Read the shapefiles and CSV files
ethnologue = gpd.read_file(poscol_path)
ethnologue_proj = ethnologue.to_crs(epsg=6933)

bii20 = rasterio.open(bii_path)
ethnologue = ethnologue.to_crs(bii20.crs)

In [22]:
# Compute area of each polygon
ethnologue_proj['area_km2'] = ethnologue_proj.geometry.area / 1e6

# Merging area back to original dataframe
ethnologue['area_km2'] = ethnologue_proj['area_km2']

In [23]:
# Compute zonal statistics (mean within each polygon)
stats = zonal_stats(ethnologue, bii_path, stats=["mean"], geojson_out=True)

In [24]:
ethnologue_bii = gpd.GeoDataFrame.from_features(stats)
ethnologue_bii.rename(columns={"mean": "bii"}, inplace=True)

print(ethnologue_bii[['bii', 'area_km2']].head())

         bii      area_km2
0  60.280536  8.056159e+06
1  44.632869  7.104573e+06
2  59.676080  6.780013e+06
3  46.753735  3.764472e+06
4  45.162215  3.257609e+06


In [25]:
# Keep only relevant columns
ethnologue_bii = ethnologue_bii[["ID","area_km2","bii"]]

# Export to CSV
ethnologue_bii.to_csv(maps_path / "BII" / "ethnologue_bii.csv", index=False)

print(f"Exported ethnologue_bii.csv")

Exported ethnologue_bii.csv
